In [ ]:
#path = r"C:\Users\adgjl\Desktop\chromedriver-win64\chromedriver.exe"
path = r"C:\Users\NT551_11TH\Desktop\chromedriver-win64\chromedriver.exe"

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import re
import time
import datetime
from bs4 import BeautifulSoup

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 타자 데이터 불러기기
batter = pd.read_excel(r"C:\Users\NT551_11TH\Desktop\ML_project\4.xlsx")
batter

,name,player_id,hand
0,Christian Encarnacion-Strand,687952,R
1,Jose Miranda,669304,R
2,Jacob Wilson,805779,R
3,José Abreu,547989,R
4,Tyler Stephenson,663886,R
...,...,...,...
163,Tyler O'Neill,641933,R
164,Korey Lee,686676,R
165,Michael Toglia,669911,S
166,Brendan Donovan,680977,L


In [ ]:
total_data = [] # 전체 데이터 리스트

In [ ]:
len(batter)

168

In [ ]:

p_name = batter.loc[:, "name"] # 선수 이름
p_id = batter.loc[:, "player_id"] # 선수 id

# 타자 수만큼
for idx in range(len(batter)):

    # id, name, year 불러와서
    pid = p_id[idx]
    pname = p_name[idx]
    year = 2023

    base_url = f"https://baseballsavant.mlb.com/illustrator?playerId={pid}&playerType=batter&name=Ha-Seong%20Kim&tabSelection=0&shape=square&shapeName=Square%20(Instagram)&subTitle=&chartType=sprayChart&chartName=Spray%20Chart&pitcherThrows=&batterStands=&plateAppearanceResults=[]&pitchTypes=[]&seasonType=[%22R%22]&pitchResults=[]&facingPlayer=[]&balls=[]&battedBallType=[]&years=[{year}]&strikes=[]&outs=[]&selectedGames=[]&homeTeam=[]&awayTeam=[]&vsTeam=[]&exitVelocityGT=0&exitVelocityLT=125&pitchVelocityGT=0&pitchVelocityLT=105&launchAngleGT=-90&launchAngleLT=90&perspective=catcher&venue=7"
    s = Service(path)
    driver = webdriver.Chrome(service = s)
    driver.get(base_url)
    time.sleep(7)
    # driver.implicitly_wait(10)

    print(f"pid : {pid}, name : {pname} start!")

    try:
        # 전체 타구 갯수 확인
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        tot_circle = len(soup.find("g", {"id": "sprayChart"}).find_all("circle"))
        print(f"Total circles: {tot_circle}")
        # 타구 수가 0이면 다음 선수로 이동
        if tot_circle == 0:
            print(f"No spray chart data for pid: {pid}, name: {pname}. Moving to next player...")
            driver.quit()

            continue  # 다음 선수로 이동
    except AttributeError:
        print(f"Spray chart data not found for pid: {pid}, name: {pname}. Moving to next player...")
        driver.quit()
        continue  # 다음 선수로 이동

    for ball in range(tot_circle):
        temp = []  # 1개 행 데이터 담을 리스트
        temp.append(pid)  # 선수 ID 추가
        temp.append(pname)
        temp.append(year)  # 연도 추가

        circle = f'//*[@id="sprayChart_{ball}"]'
        info_circle = driver.find_element(By.XPATH, circle)
        temp.append(info_circle.get_attribute("cx"))
        temp.append(info_circle.get_attribute("cy"))
        temp.append(info_circle.get_attribute("fill"))

        try:
            # 타구 클릭
            target_element = driver.find_element(By.CSS_SELECTOR, f"#sprayChart_{ball}")
            actions = ActionChains(driver)
            actions.move_to_element(target_element).click().perform()
            time.sleep(3)

            # 탭 변경
            original_tab = driver.current_window_handle
            all_tabs = driver.window_handles

            if len(all_tabs) <= 1:  # 새로운 탭이 열리지 않은 경우
                print(f"{ball + 1}번째 타구 클릭 후 새로운 탭이 열리지 않았습니다. 다음 타구로 넘어갑니다.")

                continue  # 다음 타구로 이동

            for tab in all_tabs:
                if tab != original_tab:
                    driver.switch_to.window(tab)
                    break

            # 새 탭의 URL 출력
            new_tab_url = driver.current_url
            print(f"새 탭 URL: {new_tab_url}")
            time.sleep(0.5)

            # 타구 정보 수집

            soup = BeautifulSoup(driver.page_source, 'html.parser')
            batter = soup.find("div", class_="meta-box").find_all("div", class_="mod")[1].find_all("li")[0].text.split(":")[1].strip()
            pitcher = soup.find("div", class_="meta-box").find_all("div", class_="mod")[1].find_all("li")[1].text.split(":")[1].strip()
            count = soup.find("div", class_="meta-box").find_all("div", class_="mod")[1].find_all("li")[2].text.split(":")[1].strip()
            pitch_type = soup.find("div", class_="meta-box").find_all("div", class_="mod")[1].find_all("li")[3].text.split(":")[1].strip()
            velocity = float(soup.find("div", class_="meta-box").find_all("div", class_="mod")[1].find_all("li")[4].text.split(":")[1].strip())
            exit_velocity = float(soup.find("div", class_="meta-box").find_all("div", class_="mod")[1].find_all("li")[5].text.split(":")[1].strip())
            hit_dist = int(soup.find("div", class_="meta-box").find_all("div", class_="mod")[1].find_all("li")[6].text.split(":")[1].strip())

            if '/' in soup.find("div", class_="meta-box").find_all("div", class_="mod")[1].find_all("li")[7].text.split(":")[1].strip():
                hr = soup.find("div", class_="meta-box").find_all("div", class_="mod")[1].find_all("li")[7].text.split(":")[1].strip()
                matchup = soup.find("div", class_="meta-box").find_all("div", class_="mod")[1].find_all("li")[8].text.split(":")[1].strip()
                date = soup.find("div", class_="meta-box").find_all("div", class_="mod")[1].find_all("li")[9].text.split(":")[1].strip()
            else:
                hr = ''
                matchup = soup.find("div", class_="meta-box").find_all("div", class_="mod")[1].find_all("li")[7].text.split(":")[1].strip()
                date = soup.find("div", class_="meta-box").find_all("div", class_="mod")[1].find_all("li")[8].text.split(":")[1].strip()

            temp.extend([batter, pitcher, count, pitch_type, velocity, exit_velocity, hit_dist, hr, matchup, date])

            # 날라온 공 위치
            small_box = soup.find('svg', id='svg_zone_chart').find_all("rect")
            large_box = soup.find('svg', id='svg_zone_chart').find_all("path")

            for s in range(1, 10):
                if "fill: rgb(255, 0, 0)" in small_box[s].get("style"):
                    temp.append(s)  # temp에 위치 추가
            for l in range(4):
                if large_box[l].get("fill") == "#FF0000":
                    temp.append(l + 11)  # temp에 위치 추가

            print(f"{ball + 1}번째 타구 데이터 수집 완료!")
            print(temp)

            # 전체 데이터 리스트에 추가
            total_data.append(temp)

            # 타구 선택 페이지로 이동
            main = driver.window_handles
            driver.close()
            driver.switch_to.window(main[0])
            time.sleep(0.5)

        except Exception as e:
            print(f"{ball + 1}번째 타구 처리 중 오류 발생: {e}. 다음 타구로 넘어갑니다.")
            # 현재 탭이 남아 있다면 닫고 원래 탭으로 돌아가기
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(original_tab)
            continue  # 다음 타구로 이동

df1 = pd.DataFrame(total_data,
                  columns = ["id", "name", "year", "cx", "cy", "ball_type", "batter", "pitcher", "count", "pitch_type", "velocity", "exit_velocity", "hit_dist", "hr", "matchup", "date", "zone_num"])

df1.to_excel("mlb.xlsx")


pid : 641355, name : Cody Bellinger start!
Total circles: 432
새 탭 URL: https://baseballsavant.mlb.com/sporty-videos?playId=607c2507-e5b9-437d-93da-b3c08401253c
1번째 타구 데이터 수집 완료!
[641355, 'Cody Bellinger', 2024, '375.4492682926829', '124.60439024390246', '#c2c2c2', 'Bellinger, Cody', 'Eovaldi, Nathan', '2 - 2', 'FS', 89.5, 102.4, 361, '1/30 parks', 'CHC @ TEX', '2024-03-28', 7]
새 탭 URL: https://baseballsavant.mlb.com/sporty-videos?playId=ef71cc71-be58-4d1b-8276-0d95c3588e9e
2번째 타구 데이터 수집 완료!
[641355, 'Cody Bellinger', 2024, '270.919756097561', '282.46878048780485', '#c2c2c2', 'Bellinger, Cody', 'Sánchez, Cristopher', '2 - 2', 'SL', 84.8, 84.3, 26, '', 'CHC @ PHI', '2024-09-25', 13]
새 탭 URL: https://baseballsavant.mlb.com/sporty-videos?playId=160282d6-228c-48e2-badb-573197a39381
3번째 타구 데이터 수집 완료!
[641355, 'Cody Bellinger', 2024, '319.97414634146344', '333.7490243902439', '#785EF0', 'Bellinger, Cody', 'Gil, Luis', '2 - 2', 'SL', 87.5, 94.5, 46, '', 'NYY @ CHC', '2024-09-06', 7]
새 탭 URL: h